### Zigbang 원룸 매물 데이터 수집

In [1]:
import warnings
warnings.filterwarnings('ignore')
import requests
import pandas as pd

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

In [7]:
# 1. 웹페이지 분석 : 개발자 도구 : URL
# https://apis.zigbang.com/v2/search?leaseYn=N&q=%EB%A7%9D%EC%9B%90%EB%8F%99&serviceType=%EC%9B%90%EB%A3%B8
addr = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
# 2. Request(URL) > Response(Json(str))
response = requests.get(url)

# 3. Json(str) > list, dict > 위도, 경도
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

#### 2. 위도 경도로 geohash 알아내기
- install geohash2 package

In [8]:
!pip install geohash2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15554 sha256=fd5b1c9c4fb4da02728fbcce0b0cb53347fda4237efa4550cd21090bee12d4f5
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\f6\7c\c4\1b3c6fea0ebc53bf730dc86bbee7a713d501455dfb4c1f0623
Successfully built geohash2


In [9]:
import geohash2

In [10]:
# precision : 영역크기 : 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

#### 3. geohash로 매물 아이디 가져오기

In [12]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

response = requests.get(url)
response

<Response [200]>

In [20]:
items = response.json()['items']
ids = []
for item in items:
    print(item['item_id'])
    ids.append(item['item_id'])
ids

37623131
37685329
37699264
37828417
37595863
37812771
37594373
37779193
37681821
37789981
37812120
37591840
37647361
37776332
37821042
37822875
37822954
37830915
37831419
37802231
37807450
37573837
37806009
37756478
37756629
37687435
37732891
37545432
37733547
37777449
37815927
37840494
37805662
37738304
37809180
37725362
37725378
37791522
37819640
37823879
37769944
37784321
37811134
37820390
37803194
37708036
37820398
37807105
37680236
37682630
37563947
37603729
37629225
37699830
37609079
37623279
37623344
37792632
37819910
37816837
37798113
37798222
37801140
37757747
37633675
37700428
37833446
35677284
37448728
37624965
37841157
37658808
37687107
37709024
37709031
37773811
37782834
37812725
37377665
37682567
37751948
37725986
37798348
37839779
37840783
37626753
37519972
37696725
37823914
37597484
37800296
37760927
37788461
37789081
37791583
37609792
37840360
37742550
37786434
37789526
37740035
37791557
37831773
35325320
36832429
37622957
37681902
37696432
37715279
37792647
37807349
3

[37623131,
 37685329,
 37699264,
 37828417,
 37595863,
 37812771,
 37594373,
 37779193,
 37681821,
 37789981,
 37812120,
 37591840,
 37647361,
 37776332,
 37821042,
 37822875,
 37822954,
 37830915,
 37831419,
 37802231,
 37807450,
 37573837,
 37806009,
 37756478,
 37756629,
 37687435,
 37732891,
 37545432,
 37733547,
 37777449,
 37815927,
 37840494,
 37805662,
 37738304,
 37809180,
 37725362,
 37725378,
 37791522,
 37819640,
 37823879,
 37769944,
 37784321,
 37811134,
 37820390,
 37803194,
 37708036,
 37820398,
 37807105,
 37680236,
 37682630,
 37563947,
 37603729,
 37629225,
 37699830,
 37609079,
 37623279,
 37623344,
 37792632,
 37819910,
 37816837,
 37798113,
 37798222,
 37801140,
 37757747,
 37633675,
 37700428,
 37833446,
 35677284,
 37448728,
 37624965,
 37841157,
 37658808,
 37687107,
 37709024,
 37709031,
 37773811,
 37782834,
 37812725,
 37377665,
 37682567,
 37751948,
 37725986,
 37798348,
 37839779,
 37840783,
 37626753,
 37519972,
 37696725,
 37823914,
 37597484,
 37800296,

In [21]:
len(ids), ids[:5]

(581, [37623131, 37685329, 37699264, 37828417, 37595863])

In [22]:
# listcomprehension : 간단한 for, if를 사용해서 리스트 데이터를 만들때 사용되는 문법
# [var for var in collection if condition]
ids = [item['item_id'] for item in items]
ids[:5]

[37623131, 37685329, 37699264, 37828417, 37595863]

#### 4. 매물 아이디로 매물 정보 가져오기

In [24]:
# 요청 id가 1000개를 초과하면 에러발생
url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain' : 'zigbang',
    'item_ids':ids,
}

response = requests.post(url, params)
response

<Response [200]>

In [28]:
data = response.json()['items']
df = pd.DataFrame(data)
df.tail(2)

,item_id,section_type,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_new,addressOrigin,action,contract,address,is_zzim,address1,address2,address3,item_bm_type
579,37703323,None,https://ic.zigbang.com/ic/items/37703323/1.jpg,월세,월세,5000,160,63.97,"{'m2': 63.97, 'p': '19.4'}","{'m2': 42.52, 'p': '12.9'}",...,False,"{'local1': '서울시', 'local2': '마포구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG
580,37836529,None,https://ic.zigbang.com/ic/items/37836529/1.jpg,월세,월세,5000,160,61.09,"{'m2': 61.09, 'p': '18.5'}","{'m2': 40.61, 'p': '12.3'}",...,True,"{'local1': '서울특별시', 'local2': '마포구', 'local3':...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG


In [30]:
pd.options.display.max_columns = 40

In [31]:
df.tail(2)

,item_id,section_type,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,status,tags,service_type,random_location,manage_cost,reg_date,is_new,addressOrigin,action,contract,address,is_zzim,address1,address2,address3,item_bm_type
579,37703323,None,https://ic.zigbang.com/ic/items/37703323/1.jpg,월세,월세,5000,160,63.97,"{'m2': 63.97, 'p': '19.4'}","{'m2': 42.52, 'p': '12.9'}",None,None,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,None,05,True,[추천],빌라,"{'lat': 37.56930186522451, 'lng': 126.90985826...",8,2023-08-12T09:50:21+09:00,False,"{'local1': '서울시', 'local2': '마포구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG
580,37836529,None,https://ic.zigbang.com/ic/items/37836529/1.jpg,월세,월세,5000,160,61.09,"{'m2': 61.09, 'p': '18.5'}","{'m2': 40.61, 'p': '12.3'}",None,None,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,None,04,True,[],오피스텔,"{'lat': 37.56981487490897, 'lng': 126.90922904...",8,2023-08-24T11:44:39+09:00,True,"{'local1': '서울특별시', 'local2': '마포구', 'local3':...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG


In [32]:
# pd.options.display.max_rows 

60

In [34]:
# df.columns
columns = ['item_id','sales_type', 'deposit', 'rent', 'size_m2',
          'floor', 'floor_string', 'building_floor', 'title',
          'service_type','manage_cost','address1']
df = df[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,floor_string,building_floor,title,service_type,manage_cost,address1
579,37703323,월세,5000,160,63.97,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,빌라,8,서울시 마포구 중동
580,37836529,월세,5000,160,61.09,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,오피스텔,8,서울시 마포구 중동


In [38]:
result = df.loc[df['address1'].str.contains('망원동')].reset_index(drop=True)
result

,item_id,sales_type,deposit,rent,size_m2,floor,floor_string,building_floor,title,service_type,manage_cost,address1
0,37814767,월세,2000,45,23.14,1,1,3,3000.40 가능 . 풀옵션 . 큰 원룸 . 1.5층,원룸,5,서울시 마포구 망원동
1,37811154,월세,2000,45,23.14,1,1,3,리모델링원룸 빠른입주가능,원룸,5,서울시 마포구 망원동
2,37568534,전세,23000,0,44.29,2,2,2,가성비 따듯한 감성 리모델링 투룸,빌라,3,서울시 마포구 망원동
3,37626743,전세,28000,0,65.37,2,2,5,쓰리룸 빌라전세 HUG안심대출 주차가능,빌라,0,서울시 마포구 망원동
4,37814752,전세,27000,0,52.09,4,4,5,버팀목.보증보험가능. 테라스 있는 3룸이에요,빌라,3,서울시 마포구 망원동
5,37825096,전세,27000,0,52.89,4,4,5,망원역세권 테라스가 넓은 3룸 입니다.,빌라,3,서울시 마포구 망원동
6,37796306,전세,16000,0,41.21,2,2,3,가성비 주차 동물 버팀목 가능 투룸,빌라,1,서울시 마포구 망원동
7,37666663,월세,1000,75,23.14,2,2,4,망원동 핫플 리모델링 완료후 첫 입주,원룸,5,서울시 마포구 망원동
8,37709901,월세,1000,70,23.14,4,4,4,망원 신축 첫입주 풀옵션 리모델링 평수 채광 굳굳,원룸,5,서울시 마포구 망원동
9,37741462,전세,14000,0,26.45,2,2,4,잔여세대 1개ㅣ망원역5분ㅣ올리모델링ㅣ1.5룸,원룸,5,서울시 마포구 망원동


In [44]:
# function
def oneroom(addr):

    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    geohash = geohash2.encode(lat, lng, precision=5)
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain' : 'zigbang',
        'item_ids':ids,
    }
    response = requests.post(url, params)
    
    data = response.json()['items']
    df = pd.DataFrame(data)
    result = df.loc[df['address1'].str.contains(addr)].reset_index(drop=True)
    return result[['item_id', 'title', 'deposit', 'rent', 'address1', 'size_m2', 'manage_cost']]

In [46]:
result = oneroom('마포구 합정동')
result.tail(2)

NameError: name 'requests' is not defined

In [42]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [45]:
%whos

Variable   Type        Data/Info
--------------------------------
oneroom    function    <function oneroom at 0x000001BC66A4AAC0>


In [47]:
import requests
import pandas as pd
import geohash2

In [48]:
%whos

Variable   Type        Data/Info
--------------------------------
geohash2   module      <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
oneroom    function    <function oneroom at 0x000001BC66A4AAC0>
pd         module      <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module      <module 'requests' from '<...>\\requests\\__init__.py'>


In [50]:
%%time
result = oneroom('마포구 합정동')
result.tail(2)

CPU times: total: 93.8 ms
Wall time: 494 ms


,item_id,title,deposit,rent,address1,size_m2,manage_cost
63,37470018,o합정역o 3룸 올수리 대박,3000,120,서울시 마포구 합정동,59.50,2
64,37668099,가성비 허그 LH 리모델링 투룸,20000,0,서울시 마포구 합정동,59.81,1


In [51]:
# 모듈 파일 만들기 : 변수, 함수, 클래스를 하나의 파일(.py)로 묶어서 코드 작성 실행 문법 

In [52]:
%%writefile zigbang.py
import requests
import pandas as pd
import geohash2

# function
def oneroom(addr):

    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    geohash = geohash2.encode(lat, lng, precision=5)
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain' : 'zigbang',
        'item_ids':ids,
    }
    response = requests.post(url, params)
    
    data = response.json()['items']
    df = pd.DataFrame(data)
    result = df.loc[df['address1'].str.contains(addr)].reset_index(drop=True)
    return result[['item_id', 'title', 'deposit', 'rent', 'address1', 'size_m2', 'manage_cost']]

Writing zigbang.py


In [54]:
%ls zigbang.py

 C 드라이브의 볼륨: WINDOWS
 볼륨 일련 번호: 1CE2-C115

 C:\Users\user\Web_Crawling\day1 디렉터리

2023-08-24  오후 05:00             1,092 zigbang.py
               1개 파일               1,092 바이트
               0개 디렉터리  169,887,973,376 바이트 남음
